## 1. Fast quantum simulation algorithms

While product formulas provide the most straightforward approach to Hamiltonian simulation, alternative approaches can offer improved performance. Here we begin to explore Hamiltonian simulation beyond product formulas.

### 1.1. No fast-forwarding

Before introducing improved upper bounds, we establish a limitation on the ability of quantum algorithms to simulate sparse Hamiltonians. Specifically, we show that no general procedure can simulate a sparse Hamiltonian acting for time $t$ using $o(t)$ queries [19](https://arxiv.org/abs/quant-ph/0508139).

The lower bound is based on a reduction from parity. Recall that computing the parity of $n$ bits requires $\Omega(n)$ queries. Given an input string $x \in\{0,1\}^{n}$, construct a graph on vertices $(i, b)$ for $i \in\{0,1, \ldots, n\}$ and $b \in\{0,1\}$, such that $(i-1, b)$ is adjacent to $\left(i, b \oplus x_{i}\right)$ for all $i \in\{1, \ldots, n\}$ and $b \in\{0,1\}$. This graph is the disjoint union of two paths of length $n$, and $(0,0)$ is connected to $(n, b)$ for exactly one value of $b$, namely $b=x_{1} \oplus \cdots \oplus x_{n}$, the parity of the input string. The main idea of the proof is to construct a Hamiltonian whose nonzero entries correspond to this graph, such that the dynamics for some time $t=O(n)$ map the state $|0,0\rangle$ to the state $\left|n, x_{1} \oplus \cdots \oplus x_{n}\right\rangle$. Then a simulation of the Hamiltonian dynamics for time $t$ using $o(t)$ queries would violate the parity lower bound.

The most obvious choice is to simply use the adjacency matrix of the graph as the Hamiltonian. However, then the dynamics generate a continuous-time quantum walk on a finite path, which does not reach the opposite end of the path with constant amplitude after linear time.

Instead, we choose the matrix elements of the Hamiltonian $H$ so that

$$
\langle i-1, b| H\left|i, b \oplus x_{i}\right\rangle=\sqrt{i(n-i+1)} / n
$$

Clearly, a black box for this 2-sparse Hamiltonian can be implemented using $O(1)$ queries to the black box for the input string to answer each neighbor query. The weights are chosen to reflect the transitions between column states for an unweighted hypercube. Specifically, letting $Q$ denote the adjacency matrix of the hypercube and $\left|\mathrm{wt}_{k}\right\rangle:=\binom{n}{k}^{-1 / 2} \sum_{|x|=k}|x\rangle$, we have

$$
\begin{aligned}
Q\left|\mathrm{wt}_{k}\right\rangle & =\binom{n}{k}^{-1 / 2}\left((n-k+1) \sum_{|x|=k-1}|x\rangle+(k+1) \sum_{|x|=k+1}|x\rangle\right) \\
& =\sqrt{k(n-k+1)}\left|\mathrm{wt}_{k-1}\right\rangle+\sqrt{(k+1)(n-k)}\left|\mathrm{wt}_{k+1}\right\rangle .
\end{aligned}
$$

Thus, with these weights on the edges, the dynamics behave just as the walk on the hypercube within its column subspace. In particular, since the dynamics on the hypercube map a vertex into the opposite corner in time $\pi / 2$, the chosen Hamiltonian maps $|0,0\rangle$ to $\left|n, x_{1} \oplus \cdots \oplus x_{n}\right\rangle$ in time $O(n)$.

It follows that a generic procedure for simulating sparse Hamiltonians for time $t$ must have complexity $\Omega(t)$ in general. In other words, one cannot "fast-forward" the dynamics of arbitrary Hamiltonians.

### 1.2. Quantum walk

Simulations based on product formulas have superlinear complexity. A conceptually different approach to Hamiltonian simulation uses the notion of a discrete-time quantum walk (specifically, the Szegedy framework). Here, one defines a quantum walk that is closely related to any given time-independent Hamiltonian and applies phase estimation in order to simulate the Schrödinger dynamics [30](https://arxiv.org/abs/0810.0312). This approach gives a simulation of sparse Hamiltonians acting for time $t$ with complexity $O(t)$, matching the lower bound of the previous section.

Define states

$$
\left.\left|\psi_{j}\right\rangle:=\frac{1}{\sqrt{X}} \sum_{k=1}^{N} \sqrt{H_{j k}^{*}}|j, k\rangle+\sqrt{\left.1-\frac{1}{X} \sum_{k=1}^{N}\left|H_{j k}\right| \right\rvert\, j}, N+1\right\rangle
$$

where $X \geq \max _{j} \sum_{k=1}^{N}\left|H_{j k}\right|$. Define the operators $S, T$ such that we get $T^{\dagger} S T=H / X$, so the walk has eigenvalues $e^{ \pm i \arccos (\lambda / X)}$, where $\lambda$ is an eigenvalue of $H$. The eigenvectors corresponding to these two eigenvalues can be found within the subspace $\operatorname{span}\{T|\lambda\rangle, S T|\lambda\rangle\}$, where $|\lambda\rangle$ is the eigenvector of $H$ with eigenvalue $\lambda$.

To simulate $H$ on a given input state $|\psi\rangle$, we proceed as follows:

1. Apply the isometry $T$ to produce the state $T|\psi\rangle$.
2. Perform phase estimation on the quantum walk with precision $\delta$ (to be determined).
3. Given a value approximating $\arccos (\lambda / X)$, compute an estimate of $\lambda$.
4. Introduce the phase $e^{-i \lambda t}$.
5. Uncompute the estimate of $\lambda$.
6. Invert the phase estimation procedure.
7. Apply $T^{\dagger}$ to return to a state in the original Hilbert space.

Since a step of the quantum walk can be implemented using two applications of the isometry $T$, this procedure makes $O(1 / \delta)$ calls to $T$. In turn, $T$ can be implemented using a number of queries that is polynomial in the sparsity of the Hamiltonian, so up to factors of the sparsity, the query complexity of simulation is simply $O(1 / \delta)$. Thus it remains to determine what value of $\delta$ suffices to ensure that the overall procedure reproduces the dynamics up to error at most $\epsilon$.

The details of this analysis are presented in [30](https://arxiv.org/abs/0810.0312) [20](https://arxiv.org/abs/0910.4157), but we can understand it roughly as follows. Suppose the estimate of $\arccos (\lambda / X)$ deviates from its true value by of order $\delta$. Since the cosine function has Lipschitz constant 1 (i.e., $|\cos (x+\Delta)-\cos (x)| \leq|\Delta|)$, the resulting error in the value of $\lambda / X$ is also of order $\delta$. In other words, the error in the value of $\lambda$ is of order $X \delta$. To ensure that $e^{-i \lambda t}$ deviates by at most $\epsilon$ from its true value, we take $X \delta t=\Theta(\epsilon)$, i.e., $1 / \delta=\Theta(X t / \epsilon)$. Thus we see that the complexity is linear in $t$ and polynomial in $1 / \epsilon$. Note if $H$ is $d$-sparse, then we can choose $X \leq \sqrt{d}\|H\| \leq d\|H\|_{\max }$, so the factor of $X$ just introduces polynomial overhead with respect to the sparsity.

Using a more refined implementation and analysis of this approach, one can achieve query complexity $O\left(\frac{\|H\| t}{\sqrt{\epsilon}}+d\|H\|_{\max } t\right)=O\left(d\|H\|_{\max } t / \sqrt{\epsilon}\right)$ for a $d$-sparse Hamiltonian $H[20]$.

### 1.3. Linear combinations of unitaries

While the quantum walk approach described in the previous section gives optimal complexity as a function of the simulation time $t$, its performance as a function of the allowed error $\epsilon$ is worse than using high-order product formulas. It is natural to ask how efficiently we can simulate Hamiltonian dynamics as a function of $\epsilon$, and in particular, whether we can achieve complexity poly $(\log (1 / \epsilon))$.

This can indeed be achieved by another approach to Hamiltonian simulation [21](https://arxiv.org/abs/1312.1414), [22](https://arxiv.org/abs/1412.4687), which is based on techniques for implementing linear combinations of unitary operators on a quantum computer. This approach strictly improves over direct use of product formulas, giving faster performance without the need
for higher-order formulas. The query complexity of this approach is $O\left(\tau \frac{\log (\tau / \epsilon)}{\log \log (\tau / \epsilon)}\right)$, where $\tau:=d^{2}\|H\|_{\max } t$ (with $\|H\|_{\max }$ denoting the largest magnitude of an entry of $H$ ).

Denote the Taylor series for the evolution up to time $t$, truncated at order $K$, by

$$
\tilde{U}(t):=\sum_{k=0}^{K} \frac{(-i H t)^{k}}{k!}
$$

For sufficiently large $K$, the operator $\tilde{U}(t)$ is a good approximation of $\exp (-i H t)$. Specifically, by Taylor's theorem, we have

$$
\|\tilde{U}(t)-\exp (-i H t)\| \leq \frac{\exp (\|H\| t)(\|H\| t)^{K+1}}{(K+1)!}
$$

so we can ensure that the error is at most $\epsilon$ by taking $K=O(\log (\|H\| t / \epsilon) / \log \log (\|H\| t / \epsilon))$. If we take $\|H\| t$ constant, then we get an approximation with $K=O(\log (1 / \epsilon) / \log \log (1 / \epsilon))$. If we could implement the evolution for constant time with this complexity, then by reducing the error to $\epsilon / t$, we could repeat the process $O(t)$ times and get a simulation with complexity $O(t \log (t / \epsilon) / \log \log (t / \epsilon))$ and overall error at most $\epsilon$.

Now suppose we can decompose the given Hamiltonian in the form

$$
H=\sum_{\ell=1}^{L} \alpha_{\ell} H_{\ell}
$$

for some coefficients $\alpha_{\ell} \in \mathbb{R}$, where the individual terms $H_{\ell}$ are both unitary and Hermitian. This is straightforward if $H$ is $k$-local, since we can express the local terms as linear combinations of Pauli operators. If $H$ is sparse, then such a decomposition can also be constructed efficiently [21](https://arxiv.org/abs/1312.1414).

To implement $\tilde{U}(t)$, we begin by writing it as a linear combination of unitaries, namely

$$
\begin{aligned}
\tilde{U}(t) & =\sum_{k=0}^{K} \frac{(-i H t)^{k}}{k!} \\
& =\sum_{k=0}^{K} \sum_{\ell_{1}, \ldots, \ell_{k}=1}^{L} \frac{t^{k}}{k!} \alpha_{\ell_{1}} \cdots \alpha_{\ell_{k}}(-i)^{k} H_{\ell_{1}} \cdots H_{\ell_{k}} \\
& =\sum_{j=0}^{m-1} \beta_{j} V_{j}
\end{aligned}
$$

where the $V_{j}$ are products of the form $(-i)^{k} H_{\ell_{1}} \cdots H_{\ell_{k}}$, and the $\beta_{j}$ are the corresponding coefficients.
How can we implement such a linear combination of unitaires? Let $B$ be an operation that prepares the state

$$
|\beta\rangle:=\frac{1}{\sqrt{s}} \sum_{j=0}^{m-1} \sqrt{\beta_{j}}|j\rangle
$$

from $|0\rangle$ (i.e., $B|0\rangle=|\beta\rangle$ ), where

$$
\begin{aligned}
s & :=\sum_{j=0}^{m-1}\left|\beta_{j}\right| \\
& =\sum_{k=0}^{K} \sum_{\ell_{1}, \ldots, \ell_{k}=1}^{L} \frac{t^{k}}{k!}\left|\alpha_{\ell_{1}} \cdots \alpha_{\ell_{k}}\right| \\
& =\sum_{k=0}^{K} \frac{\left.\left(t \sum_{\ell=1}^{L}\left|\alpha_{\ell}\right|\right)\right)^{k}}{k!} .
\end{aligned}
$$

Let

$$
W:=B^{\dagger} \operatorname{select}(V) B
$$

with

$$
\operatorname{select}(V):=\sum_{j=0}^{m-1}|j\rangle\langle j| \otimes V_{j}
$$

Then we have

$$
\begin{aligned}
(\langle 0| \otimes I) W(|0\rangle \otimes|\psi\rangle) & =\frac{1}{\sqrt{s}}(\langle 0| \otimes I) B^{\dagger} \operatorname{select}(V) \sum_{j} \sqrt{\beta_{j}}|j\rangle|\psi\rangle \\
& =\frac{1}{\sqrt{s}}(\langle 0| \otimes I) B^{\dagger} \sum_{j} \sqrt{\beta_{j}}|j\rangle V_{j}|\psi\rangle \\
& =\frac{1}{s} \sum_{j} \beta_{j} V_{j}|\psi\rangle \\
& =\frac{1}{s} \tilde{U}(t)|\psi\rangle
\end{aligned}
$$

In other words, if we postselect the state $W(|0\rangle \otimes|\psi\rangle)$ on having its first register in the state $|0\rangle$, we obtain the desired result. However, this postselection only succeeds with probability (approximately) $1 / \mathrm{s}^{2}$.

Considering the action of $W$ on the full space, we have

$$
W|0\rangle|\psi\rangle=\frac{1}{s}|0\rangle \otimes \tilde{U}(t)|\psi\rangle+\sqrt{1-\frac{1}{s^{2}}}|\Phi\rangle
$$

for $|\psi\rangle \in \mathcal{H}$ and some $|\Phi\rangle$ whose ancillary state is supported in the subspace orthogonal to $|0\rangle$. To boost the chance of success, we might like to apply amplitude amplification to $W$. However, the initial state $|\psi\rangle$ is unknown, so we cannot reflect about it. Fortunately, something similar can be achieved using the reflection

$$
R:=(I-2|0\rangle\langle 0|) \otimes I
$$

about the subspace with $|0\rangle$ in the first register. Specifically, letting $P:=|0\rangle\langle 0|$, we have

$$
\begin{aligned}
(\langle 0| \otimes I) W R W^{\dagger} R W(|0\rangle \otimes I)= & (\langle 0| \otimes I)\left(W W^{\dagger} W-2 W W^{\dagger} P W-2 W P W^{\dagger} W\right. \\
& \left.+4 W P W^{\dagger} P W\right)(|0\rangle \otimes I) \\
= & (\langle 0| \otimes I)\left(-3 W+4 W P W^{\dagger} P W\right)(|0\rangle \otimes I)
\end{aligned}
$$

Therefore

$$
(\langle 0| \otimes I) W R W^{\dagger} R W|0\rangle|\psi\rangle=-\frac{3}{s} \tilde{U}(t)+\frac{4}{s^{3}} \tilde{U}(t) \tilde{U}(t)^{\dagger} \tilde{U}(t)
$$

which is close to $-\left(\frac{3}{s}-\frac{4}{s^{3}}\right) \tilde{U}(t)$ since $\tilde{U}(t)$ is close to unitary. In particular, if $s=2$ then this process boosts the amplitude from $1 / 2$ to 1 , analogous to Grover search with a single marked item out of 4 . For the purpose of Hamiltonian simulation, we can choose the parameters such that a single segment of the evolution has this value of $s$, and we repeat the process as many times as necessary to simulate the full evolution.

More generally, the operation $W R W^{\dagger} R W$ is analogous to the Grover iterate, and it can be applied many times to boost the amplitude for success from something small to a value close to 1 . Using this oblivious amplitude amplification, a general linear combination of unitaries can be implemented with complexity $O(1 / s)$.

